In [1]:
import tensorflow as tf
import numpy as np
import copy

In [2]:
# Import data
dataset = np.genfromtxt("../train.csv", delimiter=',')

In [3]:
test = np.genfromtxt("../test0.csv",delimiter=',')
test_eval = np.genfromtxt("../test0_eval.csv",delimiter=',')

In [5]:
print dataset.shape
print test.shape

(50282, 438)
(1781, 438)


In [6]:
# Parameters
learning_rate = 0.001
display_step = 10
training_iters = 40000
sess.run(init)
step = 1
batch_size = 50
sampling_ratio = 0.3

In [7]:
# Network Parameters
n_input = 429 # data input (img shape: 13*33)
n_classes = 98 # total classes
dropout = 0.8 # Dropout, probability to keep units

In [8]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [9]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x,ks=3,k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, ks, ks, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 33, 13, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, ks=3, k=2)
    
    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, ks=3, k=2)
    
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, ks=3, k=2)
    
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Fully connected layer
    fc2 = tf.add(tf.matmul(fc1, weights['wd2']), biases['bd2'])
    fc2 = tf.nn.relu(fc2)
    fc2 = tf.nn.dropout(fc2, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc2, weights['out']), biases['out'])
    out = tf.nn.dropout(out, dropout)
    return out

In [10]:
# Store layers weight & bias
weights = {
    # 3x3 conv, 1 input, 64 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, 1, 64])),
    # 3x3 conv, 64 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([3, 3, 64, 64])),
    # 3x3 conv, 64 inputs, 128 outputs
    'wc3': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    # fully connected,  inputs 1280, 3072 outputs
    'wd1': tf.Variable(tf.random_normal([5*2*128, 3072])),
    # fully connected, inputs 3072, 2048 outputs
    'wd2': tf.Variable(tf.random_normal([3072, 2048])),
    # 2048 inputs, 98 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([2048, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bd1': tf.Variable(tf.random_normal([3072])),
    'bd2': tf.Variable(tf.random_normal([2048])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [11]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

In [12]:
# def k_fold_cross_validation(data, K, randomise = True):
#     d = copy.deepcopy(data)
#     np.random.shuffle(d)
#     size = d.shape[0]
#     hold_out_size = int(size/K)
#     for k in range(K):
#         validation = d[(hold_out_size*k):(hold_out_size*(k+1)),:]
#         if(k == 0):
#             training = d[(hold_out_size*(k+1)):,:]
#         else:
#             training = np.append(d[(hold_out_size*(k+1)):,:],d[:(hold_out_size*k),:],axis=0)
#         yield training,validation

In [13]:
def next_batch(training, batch_size,iteration = -1):
    train_example, train_label = data_process(training)
    total_batch = int(training.shape[0]/batch_size)
    if(iteration > 0 & iteration < total_batch):
        total_batch = iteration
    for i in range(total_batch):
        low = i*batch_size
        high = (i+1)*batch_size
        yield train_example[low:high,:],train_label[low:high,:]

In [14]:
def data_process(d, input_size = 429, class_size = 98):
    size = d.shape[0]
    example = d[:,9:]
    label = d[:, 1:2]
    label_matrix = np.zeros((size,class_size))
    for i in range(size):
        label_matrix[i][int(label[i])] = 1
    return example, label_matrix

In [15]:
def next_random_batch(data, batch_size, Split=True):
    d = copy.deepcopy(data)
    np.random.shuffle(d)
    low = 0
    high = batch_size
    if(Split == False):
        return data[low:high,:]
    example, label = data_process(d)
    return example[low:high,:],label[low:high,:]

In [16]:
t_example, t_label = data_process(test_eval)
result_note = np.zeros([0,98])

In [ ]:
with tf.Session() as sess:
    
    training = next_random_batch(dataset,int(dataset.shape[0]*sampling_ratio),Split=False)
    flag = False
    
    while step * batch_size < training_iters:
        test_acc = 0
        for  _batch_xs, _batch_ys in next_batch(training,batch_size):
        # Fit training using batch data
            sess.run(optimizer, feed_dict={x: _batch_xs, y: _batch_ys, keep_prob: dropout})
            acc = 0
            if step % display_step == 0:
            # Calculate batch accuracy
                acc = sess.run(accuracy, feed_dict={x: _batch_xs, y: _batch_ys, keep_prob: 1.})
            # Calculate batch loss
                loss = sess.run(cost, feed_dict={x: _batch_xs, y: _batch_ys, keep_prob: 1.})
                test_acc = sess.run(accuracy, feed_dict={x: t_example, y: t_label, keep_prob: 1.})
                print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)+ ", Testing Accuracy= " + "{:.5f}".format(test_acc)
            
            if(step * batch_size >= training_iters):
                flag = True
                break;
            step += 1
        if(flag):
            break;
    print "Optimization Finished!"
    print "Testing Accuracy:", sess.run(accuracy, feed_dict={x: t_example, y: t_label, keep_prob: 1.})
    
    test_example, test_label = data_process(test)
    result_note = sess.run(pred, feed_dict={x:test_example,keep_prob: 1.})
    print "Done!"

In [ ]:
#write logits values into file
np.savetxt("../DeepCNN_train0_1.csv",result_note,delimiter=",")

In [ ]:
# output file
result = np.argmax(result_note,axis=1).reshape((test.shape[0],1))
output = np.zeros((test.shape[0],2))
output[:,0:1] = test[:,0:1]
output[:,1:] = result
np.savetxt("../DeepCNN0.csv",output,fmt='%d',delimiter=",",header='Id,Character')